# Analysis and Visualization of Complex Agro-Environmental Data
---
### Exercise #6 - correction

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import seaborn as sns # For plotting
import matplotlib.pyplot as plt # For showing plots
import scipy.stats as sts
import scikit_posthocs as sp
import statsmodels.stats as stm
from statsmodels.graphics.gofplots import qqplot

##### Exercise 6.1 Using the EFIplus_medit.zip dataset, test if the frequency of sites with presence and absence of Salmo trutta fario (Brown Trout) are independent from the country. Please state which is/are the null hypothesis of your test(s). You may try to produce an alluvial plot!

In [ ]:
df = pd.read_csv('../examples/EFIplus_medit.zip',compression='zip', sep=";")

In [ ]:
# clean up the dataset to remove unnecessary columns (eg. REG) 
df.drop(df.iloc[:,5:15], axis=1, inplace=True)

# let's rename some columns so that they make sense
df.rename(columns={'Sum of Run1_number_all':'Total_fish_individuals'}, inplace=True) # inplace="True" means that df will be updated

# for sake of consistency, let's also make all column labels of type string
df.columns = list(map(str, df.columns))

In [ ]:
# produce contingency table of Country and Samo trutta fario.
cdf = pd.crosstab(index=df['Country'], columns=df['Salmo trutta fario'])
print(cdf)

In [ ]:
# Chi-square test of independency
stat, p, dof, expected = sts.chi2_contingency(cdf)
print('df=%d' % dof)
print('expected values:')
print(expected)

# Alternative 1: interpret based on test-statistic
prob=0.95
critical = sts.chi2.ppf(prob, dof)
print('critical=%.3f, stat=%.3f' % (critical, stat))
if abs(stat) >= critical:
 print('stat > critical => reject H0 that variables are independent')
else:
 print('stat < critical => fail to reject H0 that variables are independent')

 # Alternative 2: interpret based on p-value
alpha = 0.05
print('significance=%.2f, p=%.3f' % (alpha, p))
if p <= alpha:
 print('reject H0 that variables are independent')
else:
 print('fail to reject H0 that variables are independent')

##### Exercise 6.2 Run the non-parametric equivalent of the test you used in exercise 5.3 and compare with the ANOVA test (5.2: Test whether there are differences in the mean elevation in the upstream catchment (Elevation_mean_catch) among the eight most sampled catchments. For which pairs of catchments are these diferences significant? Please state which is/are the null hypothesis of your test(s)).


In [ ]:
# Remove rows with at least one missing value
df.dropna(subset=['Elevation_mean_catch'], inplace=True)

In [ ]:
# Run histogram
sns.histplot(df['Elevation_mean_catch'])
plt.show()

# Run qq-plot
from statsmodels.graphics.gofplots import qqplot

qqplot(pd.Series(df['Elevation_mean_catch']), line='q')
plt.show()

In [ ]:
catchment_count = pd.crosstab(index = df['Catchment_name'], columns='count')
catch_top8 = catchment_count.sort_values(by=['count'], ascending=False).head(8).index.to_list()
dfsub = df[df.Catchment_name.isin(catch_top8)]

In [ ]:
# Boxplots
plt.figure(figsize = (10,6))
sns.boxplot(data=dfsub, x='Catchment_name', y='Elevation_mean_catch')

In [ ]:
# Run non-parametric equivalent to one-way ANOVA - Kruskal-Walis test

sample1 = df[(df['Catchment_name']=='Tejo')]['Elevation_mean_catch']
sample2 = df[(df['Catchment_name']=='Douro')]['Elevation_mean_catch']
sample3 = df[(df['Catchment_name']=='Ebro')]['Elevation_mean_catch']
sample4 = df[(df['Catchment_name']=='Cantabrica')]['Elevation_mean_catch']
sample5 = df[(df['Catchment_name']=='Guadia')]['Elevation_mean_catch']
sample6 = df[(df['Catchment_name']=='Galiza-Norte')]['Elevation_mean_catch']
sample7 = df[(df['Catchment_name']=='Minho')]['Elevation_mean_catch']
sample8 = df[(df['Catchment_name']=='Catala')]['Elevation_mean_catch']


stat, p = sts.kruskal(sample1, sample2, sample3, sample4, sample5, sample6, sample7, sample8)
print('F-statistics=%.3f, p=%.6f' % (stat, p))

alpha=0.05

if p <= alpha:
 print('reject H0 that median elevation values are equal among catchments')
else:
 print('fail to reject H0 that median elevation values are equal among catchments')

# Compute median values of Mean elevation for each catchment
dfsub[['Elevation_mean_catch','Catchment_name']].groupby('Catchment_name').median()


In [ ]:
# Run Dunn test
sp.posthoc_dunn(a=dfsub, val_col='Elevation_mean_catch', group_col= 'Catchment_name', p_adjust = 'bonferroni')

In [ ]:
# ANOVA

import statsmodels.api as sm
from statsmodels.formula.api import ols

mod = ols('Elevation_mean_catch ~ Catchment_name',
                data=dfsub).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2) # typ is the type of anova type to perform ('I','II' or 'III' = 1,2,3)
print(aov_table) # provides the usual ANOVA table

alpha=0.05
p=aov_table['PR(>F)'][0]

if p <= alpha:
 print('reject H0 that mean elevation values are equal among catchments')
else:
 print('fail to reject H0 that mean elevation values are equal among catchments')

# compute mean elevation for eacch catchment
dfsub[['Elevation_mean_catch','Catchment_name']].groupby('Catchment_name').mean()


In [ ]:
# Multiple comparisons - perform Tukey's test 
tukey = stm.multicomp.pairwise_tukeyhsd(endog=dfsub['Elevation_mean_catch'],
                          groups=dfsub['Catchment_name'],
                          alpha=0.05)
#display results
print(tukey)

##### Exercise 6.3 Using the winequality_red.csv file in the examples folder of the github repository, test which wine parameters discriminate the best between wine quality scores categorized into two classes using value 5 as the threshold value (quality>5=“good” and quality<5=“bad”).

In [ ]:
df_wine = pd.read_csv('../examples/winequality_red.csv')
df_wine.info()

In [ ]:
df_wine['quality'].head(30)

In [ ]:
# Discretize quality values into "good" and "bad"
df_wine['quality_class'] = pd.cut(x=df_wine['quality'], bins=[0, 5, 10], labels=["bad","good"]) # NOTE: the bins intervals are closed to the left (])
pd.crosstab(index = df_wine['quality_class'], columns='count') # sample sizes are relatively balanced


In [ ]:
g = df_wine.iloc[:,0:11].plot.box(subplots=True, legend=True, layout=(4, 3), figsize=(10,10))
plt.show()

In [ ]:
# Too many outliers => better to run a Mann-Whitney U test (two-tailed) for each variable

output_table = [] # define list of results

# run loop to test for each variable
for var in df_wine.columns[0:11]:
    bad = df_wine[df_wine['quality_class'] == 'bad'][var]
    good = df_wine[df_wine['quality_class'] == 'good'][var]
    stat, p = sts.mannwhitneyu(good, bad) # run Mann-Whitney U test
    output_table.append({'Variable': var,
                        'U statistic': stat,
                        'p-value': p}) # Append outputs

print(pd.DataFrame(output_table))